In [19]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from google.cloud import bigquery

In [20]:
# Télécharger la liste des tickers du S&P 500
def get_sp500_tickers():
    """Récupère la liste des tickers du S&P 500 via Wikipedia."""
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url, header=0)
    sp500_table = table[0]  # La table principale est la première
    return sp500_table["Symbol"].tolist()

In [ ]:
# Télécharger les données du jour précédent
def download_previous_day_data(tickers):
    """
    Télécharge les données daily pour les tickers du jour précédent.
    """
    # start_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    # end_date = (datetime.now() - timedelta(days=0)).strftime("%Y-%m-%d")

    current_date = datetime.now() - timedelta(days=1)
    while not pd.Timestamp(current_date).isoweekday() in range(
        1, 6
    ):  # Lundi (1) à Vendredi (5)
        current_date -= timedelta(days=1)

    start_date = current_date.strftime("%Y-%m-%d")
    end_date = (current_date + timedelta(days=1)).strftime("%Y-%m-%d")

    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date,
        end=end_date,
        interval="1d",
        group_by="ticker",
        threads=True,
    )
    if data.empty:
        print(f"Aucune donnée disponible pour {start_date}.")
        return pd.DataFrame()  # Retourne un DataFrame vide si aucune donnée
    # Restructurer les données
    data = data.stack(level=0, future_stack=True).reset_index()
    data.columns = [
        "Date",
        "Ticker",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
    ]
    return data

In [ ]:
# Charger les données dans BigQuery
def load_data_to_bigquery(df, table_id, project_id):
    """
    Charge un DataFrame dans une table BigQuery.

    Arguments :
        df : pandas.DataFrame contenant les données à charger.
        table_id : ID complet de la table BigQuery (ex : dataset.table_name).
        project_id : ID du projet Google Cloud.
    """
    client = bigquery.Client(project=project_id)

    # Convertir les données au format BigQuery
    job = client.load_table_from_dataframe(df, table_id)

    # Attendre la fin du job
    job.result()
    print(f"Les données ont été chargées dans {table_id}")


# Charger les données dans une table temporaire
def load_to_temp_table(client, df, temp_table_id):
    """
    Charge les données dans une table temporaire BigQuery.
    """
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  # Écrase les données existantes
    )
    job = client.load_table_from_dataframe(df, temp_table_id, job_config=job_config)
    job.result()
    print(f"Données chargées dans la table temporaire {temp_table_id}")


# Fusionner les données avec la table principale
def merge_into_main_table(client, temp_table_id, main_table_id):
    """
    Fusionne les données de la table temporaire dans la table principale.
    """
    query = f"""
    MERGE `{main_table_id}` AS main
    USING `{temp_table_id}` AS temp
    ON main.Date = temp.Date AND main.Ticker = temp.Ticker
    WHEN NOT MATCHED THEN
      INSERT (Date, Ticker, Open, High, Low, Close, `Adj Close`, Volume)  -- `Adj Close` avec un espace
      VALUES (temp.Date, temp.Ticker, temp.Open, temp.High, temp.Low, temp.Close, temp.`Adj Close`, temp.Volume)
    """
    job = client.query(query)
    job.result()
    print(f"Données fusionnées dans la table principale {main_table_id}")

In [ ]:
def add_daily():
    # Étape 1 : Configuration
    PROJECT_ID = "quant-dev-442615"
    DATASET_ID = "financial_data"
    MAIN_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.sp500_data"
    TEMP_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.temp_sp500_data"

    # Récupérer les tickers
    print("Téléchargement des tickers du S&P 500...")
    tickers = get_sp500_tickers()

    # Étape 2 : Télécharger les données
    print("Téléchargement des données du jour précédent...")
    sp500_data = download_previous_day_data(tickers)
    if sp500_data.empty:
        print("Aucune donnée disponible. Fin de l'exécution.")
        return  # Arrête l'exécution si aucune donnée n'est disponible
    sp500_data["Date"] = pd.to_datetime(sp500_data["Date"]).dt.date
    # Convertir la colonne Volume en entier
    sp500_data["Volume"] = sp500_data["Volume"].fillna(0).astype(int)

    # # Étape 3 : Charger dans BigQuery
    # client = bigquery.Client(project=PROJECT_ID)
    # print("Chargement des données dans une table temporaire...")
    # load_to_temp_table(client, sp500_data, TEMP_TABLE_ID)

    # # Étape 4 : Fusionner avec la table principale
    # print("Fusion des données avec la table principale...")
    # merge_into_main_table(client, TEMP_TABLE_ID, MAIN_TABLE_ID)

In [12]:
PROJECT_ID = "quant-dev-442615"
DATASET_ID = "financial_data"
MAIN_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.sp500_data"
TEMP_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.temp_sp500_data"

# Récupérer les tickers
print("Téléchargement des tickers du S&P 500...")
tickers = get_sp500_tickers()

# Étape 2 : Télécharger les données
print("Téléchargement des données du jour précédent...")
sp500_data = download_previous_day_data(tickers)
sp500_data["Date"] = pd.to_datetime(sp500_data["Date"]).dt.date
# Convertir la colonne Volume en entier
sp500_data["Volume"] = sp500_data["Volume"].fillna(0).astype(int)

Téléchargement des tickers du S&P 500...
Téléchargement des données du jour précédent...


[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-11-26 -> 2024-11-27)')


In [15]:
sp500_data.size

4024

In [18]:
total_bytes = sp500_data.memory_usage(deep=True).sum()
print(f"Poids total : {total_bytes} octets")

# Convertir en kilooctets ou mégaoctets
total_kb = total_bytes / 1024
print(f"Poids total : {total_kb} Ko")

total_mb = total_kb / 1024
print(f"Poids total : {total_mb} Mo")

Poids total : 72651 octets
Poids total : 70.9482421875 Ko
Poids total : 0.06928539276123047 Mo


In [21]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from google.cloud import bigquery


# Télécharger la liste des tickers du S&P 500
def get_sp500_tickers():
    """Récupère la liste des tickers du S&P 500 via Wikipedia."""
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url, header=0)
    sp500_table = table[0]  # La table principale est la première
    return sp500_table["Symbol"].tolist()


# Télécharger les données pour tous les tickers
def download_sp500_data(
    tickers, start_date="2000-01-01", end_date=None
) -> pd.DataFrame:
    """
    Télécharge les données daily de tous les tickers du S&P 500.
    """
    # Obtenir les données via yfinance
    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date,
        end=end_date,
        interval="1d",
        group_by="ticker",  # Organise les données par ticker
        threads=True,  # Téléchargement parallèle
    )

    data = data.stack(
        level=0, future_stack=True
    ).reset_index()  # Rendre les tickers une colonne
    data.columns = [
        "Date",
        "Ticker",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
    ]

    return data


def fill_table():
    # Étape 1 : Récupérer les tickers
    print("Téléchargement des tickers du S&P 500...")
    sp500_tickers = get_sp500_tickers()
    print(f"Nombre de tickers récupérés : {len(sp500_tickers)}")

    # Étape 2 : Télécharger les données
    print("Téléchargement des données journalières...")
    sp500_data = download_sp500_data(sp500_tickers, start_date="2000-01-01")
    sp500_data.to_csv("./sp500_data.csv", index=False)


fill_table()

Téléchargement des tickers du S&P 500...
Nombre de tickers récupérés : 503
Téléchargement des données journalières...


[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2000-01-01 -> 2024-11-30)')


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime


# Télécharger la liste des tickers du S&P 500
def get_sp500_tickers():
    """Récupère la liste des tickers du S&P 500 via Wikipedia."""
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url, header=0)
    sp500_table = table[0]  # La table principale est la première
    return sp500_table["Symbol"].tolist()


# Télécharger les données pour une liste de tickers en plusieurs lots
def download_sp500_data(
    tickers, start_date="2000-01-01", end_date=None, batch_size=10
) -> pd.DataFrame:
    """
    Télécharge les données daily de tous les tickers du S&P 500 en petits lots.
    """
    if end_date is None:
        end_date = datetime.now().strftime("%Y-%m-%d")

    all_data = []

    # Diviser les tickers en lots pour éviter les limitations de yfinance
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i : i + batch_size]
        print(f"Téléchargement du lot {i // batch_size + 1} : {batch}")
        data = yf.download(
            tickers=" ".join(batch),
            start=start_date,
            end=end_date,
            interval="1d",
            group_by="ticker",  # Organise les données par ticker
            threads=True,  # Téléchargement parallèle
        )

        # Reformatage des données pour fusionner tous les lots
        data = data.stack(level=0, dropna=True).reset_index()
        data.columns = [
            "Date",
            "Ticker",
            "Open",
            "High",
            "Low",
            "Close",
            "Adj Close",
            "Volume",
        ]
        all_data.append(data)

    # Concaténer toutes les données en un seul DataFrame
    return pd.concat(all_data, ignore_index=True)


def fill_table():
    # Étape 1 : Récupérer les tickers
    print("Téléchargement des tickers du S&P 500...")
    sp500_tickers = get_sp500_tickers()
    print(f"Nombre de tickers récupérés : {len(sp500_tickers)}")

    # Étape 2 : Télécharger les données en lots
    print("Téléchargement des données journalières...")
    sp500_data = download_sp500_data(sp500_tickers, start_date="2000-01-01")
    print(f"Données téléchargées pour {len(sp500_data)} lignes.")

    # Sauvegarde en fichier CSV
    sp500_data.to_csv("./sp500_data.csv", index=False)
    print("Données sauvegardées dans sp500_data.csv")


# Exécuter le script
fill_table()

Téléchargement des tickers du S&P 500...
Nombre de tickers récupérés : 503
Téléchargement des données journalières...
Téléchargement du lot 1 : ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


[*********************100%***********************]  10 of 10 completed
C:\Users\Hadrien\AppData\Local\Temp\ipykernel_18548\4072023076.py:41: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0, dropna=True).reset_index()


Téléchargement du lot 2 : ['APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL']


[*********************100%***********************]  10 of 10 completed
C:\Users\Hadrien\AppData\Local\Temp\ipykernel_18548\4072023076.py:41: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0, dropna=True).reset_index()
[                       0%                       ]

Téléchargement du lot 3 : ['GOOG', 'MO', 'AMZN', 'AMCR', 'AMTM', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT']


...

Téléchargement du lot 50 : ['WELL', 'WST', 'WDC', 'WY', 'WMB', 'WTW', 'WYNN', 'XEL', 'XYL', 'YUM']


[*********************100%***********************]  10 of 10 completed
C:\Users\Hadrien\AppData\Local\Temp\ipykernel_18548\4072023076.py:41: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0, dropna=True).reset_index()


Téléchargement du lot 51 : ['ZBRA', 'ZBH', 'ZTS']


[*********************100%***********************]  3 of 3 completed
C:\Users\Hadrien\AppData\Local\Temp\ipykernel_18548\4072023076.py:41: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0, dropna=True).reset_index()


Données téléchargées pour 2767480 lignes.
Données sauvegardées dans sp500_data.csv
